In [1]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *
import pyvista as pv
from matplotlib.colors import ListedColormap

In [2]:
earthquake_data = pv.read('earthquake_data.vtk')
earthquake_data

PolyData (0x20173518700)
  N Cells:    1
  N Points:   31909
  N Strips:   0
  X Bounds:   -1.272e+07, -1.211e+07
  Y Bounds:   4.404e+06, 5.236e+06
  Z Bounds:   -9.990e+00, 6.628e+00
  N Arrays:   31

In [20]:
def get_mesh_by_year(year):
    lat_lng = []
    mags = []
    for i in range(len(earthquake_data.point_data['Lat'])):
        if earthquake_data.point_data['Year'][i] == year:
            lat_lng.append([earthquake_data.point_data['Long'][i], earthquake_data.point_data['Lat'][i], 
                        0])
            mags.append(earthquake_data.point_data['Mag'][i])
            
    my_mesh = pv.UnstructuredGrid()
    my_mesh.points = lat_lng
    my_mesh.point_data["Mag"] = mags
    return my_mesh

utah_corners = np_points = np.array([[-114.04172399259161, 41.99372594774304, 0.0], 
                      [-111.0466887774167, 42.0015686697003, 0.0],
                      [-111.04673083487624, 40.997968805895084, 0.0],
                      [-109.05007687780909, 41.000660502664616, 0.0],
                      [-109.04522189716712, 36.999084296202696, 0.0],
                      [-114.05060037628137, 37.00039575681357, 0.0]])

In [24]:
def get_cmap(mesh):
    c1 = np.array([0.99, 0.0, 0.0])
    c2 = np.array([0.0, 0.99, 0.0])
    c3 = np.array([0.0, 0.0, 0.99])
    c4 = np.array([0.99, 0.99, 0.0])
    c5 = np.array([0.99, 0.0, 0.99])
    c6 = np.array([0.0, 0.99, 0.99])

    mapping = np.linspace(my_mesh.point_data['Mag'].min(), my_mesh.point_data['Mag'].max(), 256)
    mapping
    new_colors = np.empty((256, 3))
    new_colors[mapping >= 8.0] = c1
    new_colors[mapping < 7.9] = c2
    new_colors[mapping < 6.9] = c3
    new_colors[mapping < 6.0] = c4
    new_colors[mapping < 6.0] = c5
    new_colors[mapping < 2.5] = c6

    mag_classification_cmap = ListedColormap(new_colors)
    mag_classification_cmap

    lut = pv.LookupTable(cmap=mag_classification_cmap)
    return lut

In [ ]:
my_mesh = get_mesh_by_year(2016)

In [18]:
pl = pv.Plotter()

outline_mesh = pv.PolyData(utah_corners)
pl.add_mesh(outline_mesh, point_size=15)

poly_line = pv.MultipleLines(points=utah_corners)
pl.add_mesh(poly_line, line_width=2)

line = pv.Line(pointa=utah_corners[0], pointb=utah_corners[5])
pl.add_mesh(line, line_width=2, color='white')

pl.add_mesh(my_mesh, point_size=2)

ball = pv.Sphere(radius=0.05, theta_resolution=35, phi_resolution=35)
mag_glyphs = my_mesh.glyph(geom=ball, orient=False, scale="Mag")
pl.add_mesh(mag_glyphs, cmap=lut)

legend_entries = []
legend_entries.append(['Usually not felt', c6])
legend_entries.append(['Minor damage', c5])
legend_entries.append(['Slight damage', c4])
legend_entries.append(['Lots of damage', c3])
legend_entries.append(['Serious damage', c2])
legend_entries.append(['Destroys entire communities', c1])
pl.add_legend(legend_entries)

pl.show(cpos='xy')

Widget(value="<iframe src='http://localhost:55954/index.html?ui=P_0x2010d346280_7&reconnect=auto' style='width…